# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install ..

Found existing installation: todd-ai 0.3.0
Uninstalling todd-ai-0.3.0:
  Successfully uninstalled todd-ai-0.3.0
Note: you may need to restart the kernel to use updated packages.
Processing /Users/bytedance/Developer/todd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for todd-ai: filename=todd_ai-0.3.0-py3-none-any.whl size=98583 sha256=e8e9a751eae5d29a5e9b018697eedfbdc77f75e63479f4cd3b10f2a40b7317c5
  Stored in directory: /private/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/pip-ephem-wheel-cache-zfl5ka5x/wheels/15/ef/5a/9fc12e257ce5cef16b333a2ed6c992ff9cbcc9167f7199e6ac
Successfully built todd-ai

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pathlib
import tempfile
from pprint import pprint
from typing import Any, NoReturn, TypedDict, cast

import todd
import torch
import torch.nn.functional as F
import torch.utils.data

Memo = dict[str, Any]

/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[2023-07-21 11:14:41,671 27580:140704357856832][patches.py:72 todd <module>] INFO: `ipdb` is installed. Using it for debugging.


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register()
class RunnerModel(todd.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

### Runners

In [7]:
class RunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        y: torch.Tensor = self._strategy.model(batch['x'])
        loss = F.l1_loss(y, batch['y'])
        memo['loss'] = loss
        if 'log' in memo:
            memo['log']['loss'] = f'{loss.item():.3f}'
        return memo

In [8]:
class TrainerMixin(RunnerMixin):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        memo = super()._run_iter(batch, memo)
        if 'log' in memo:
            model = cast(RunnerModel, self._strategy.module)
            memo['log']['weight'] = f'{model.weight.item():.3f}'
            memo['log']['batch'] = str(batch)
        return memo

In [9]:
@todd.RunnerRegistry.register()
class CustomValidator(RunnerMixin, todd.runners.Validator):
    pass

In [10]:
@todd.RunnerRegistry.register()
class CustomIterBasedTrainer(TrainerMixin, todd.runners.IterBasedTrainer):
    pass

In [11]:
@todd.RunnerRegistry.register()
class CustomEpochBasedTrainer(TrainerMixin, todd.runners.EpochBasedTrainer):
    pass

## Validators

In [12]:
validator_demo = todd.Config(
    type='CustomValidator',
    name='custom_validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=dict(type='LogCallback', interval=5),
)

In [13]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-07-21 11:14:42,319 27580:140704357856832][runners.py:61 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:42,322 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-21 11:14:42,324 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-21 11:14:42,326 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-21 11:14:42,327 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp1fhfolu6
└── custom_validator

2 directories, 0 files


## Trainers

In [14]:
trainer_demo = validator_demo.copy()
trainer_demo.pop('type')
trainer_demo.dataloader = todd.Config(
    batch_size=2,
    shuffle=True,
    dataset=dict(type='RunnerDataset', n=67),
)
trainer_demo.optimizer = todd.Config(type='SGD', lr=0.005)


### Iteration Based

In [15]:
iter_based_trainer_demo = trainer_demo.copy()
iter_based_trainer_demo.type = 'CustomIterBasedTrainer'
iter_based_trainer_demo.name = 'custom_iter_based_trainer'
iter_based_trainer_demo.iters = 53

In [16]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        iter_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-21 11:14:42,658 27580:140704357856832][runners.py:61 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:42,661 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=93.000 weight=0.000 batch={'x': tensor([66, 27]), 'y': tensor([132,  54])}
[2023-07-21 11:14:42,663 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=47.000 weight=0.000 batch={'x': tensor([35, 12]), 'y': tensor([70, 24])}
[2023-07-21 11:14:42,665 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=105.000 weight=0.000 batch={'x': tensor([56, 49]), 'y': tensor([112,  98])}
[2023-07-21 11:14:42,667 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [20/

### Epoch Based

In [17]:
epoch_based_trainer_demo = trainer_demo.copy()
epoch_based_trainer_demo.type = 'CustomEpochBasedTrainer'
epoch_based_trainer_demo.name = 'custom_epoch_based_trainer'
epoch_based_trainer_demo.epochs = 3

In [18]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        epoch_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-21 11:14:42,693 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:42,694 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-21 11:14:42,696 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=32.000 weight=0.000 batch={'x': tensor([11, 21]), 'y': tensor([22, 42])}
[2023-07-21 11:14:42,698 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=22.000 weight=0.000 batch={'x': tensor([10, 12]), 'y': tensor([20, 24])}
[2023-07-21 11:14:42,700 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=62.000 weight=0.000 batch={'x': tensor([47, 15]), 'y': tenso

## Callbacks

### Log

In [19]:
log_callback_demo = validator_demo.copy()
log_callback = log_callback_demo.callbacks
log_callback.with_file_handler = True
log_callback_demo.callbacks = [log_callback]

In [20]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        log_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-07-21 11:14:42,759 27580:140704357856832][runners.py:61 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:42,762 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-21 11:14:42,764 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-21 11:14:42,767 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-21 11:14:42,768 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpd2q1bbyu
└── custom_validator
    └── 2023-07-21T11-14-42_759338-08-00.log

2 directories, 1 file

[2023-07-21 11:14:42,759 27580:140704357856832][runners.py:61 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:42,762 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-21 11:14:42,764 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-21 11:14:42,767 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-21 11:14:42,768 27580:140704357856832][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000


### Optimize

In [21]:
optimize_callback_demo = iter_based_trainer_demo.copy()
optimize_callback = todd.Config(type='OptimizeCallback')
optimize_callback_demo.callbacks = [optimize_callback, log_callback]

In [22]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        optimize_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-21 11:14:43,361 27580:140704357856832][runners.py:61 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:43,366 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=47.475 weight=0.945 batch={'x': tensor([27, 63]), 'y': tensor([ 54, 126])}
[2023-07-21 11:14:43,369 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=4.760 weight=1.860 batch={'x': tensor([32, 36]), 'y': tensor([64, 72])}
[2023-07-21 11:14:43,373 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=2.362 weight=1.925 batch={'x': tensor([ 3, 60]), 'y': tensor([  6, 120])}
[2023-07-21 11:14:43,376 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [20/53]

### Learning Rate Schedule

In [23]:
lr_schedule_callback_demo = iter_based_trainer_demo.copy()
lr_schedule_callback = todd.Config(
    type='LRScheduleCallback',
    lr_scheduler=dict(type='LinearLR', total_iters=10),
)
lr_schedule_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_callback,
    log_callback,
]


In [24]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-21 11:14:43,413 27580:140704357856832][runners.py:61 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:43,418 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=54.491 weight=0.297 batch={'x': tensor([37, 27]), 'y': tensor([74, 54])} lr=['3.333e-03']
[2023-07-21 11:14:43,421 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=53.707 weight=0.869 batch={'x': tensor([51, 44]), 'y': tensor([102,  88])} lr=['5.000e-03']
[2023-07-21 11:14:43,424 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=0.711 weight=1.921 batch={'x': tensor([ 1, 17]), 'y': tensor([ 2, 34])} lr=['5.000e-03']
[2023-07-21 11:14:43,428 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_ite

In [25]:
lr_schedule_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
lr_schedule_by_epoch_callback = lr_schedule_callback.copy()
lr_schedule_by_epoch_callback.by_epoch = True
lr_schedule_by_epoch_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_by_epoch_callback,
    log_callback,
]


In [26]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_by_epoch_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-21 11:14:43,467 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:43,468 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-21 11:14:43,472 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=49.275 weight=0.175 batch={'x': tensor([22, 32]), 'y': tensor([44, 64])} lr=['1.667e-03']
[2023-07-21 11:14:43,475 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=51.841 weight=0.453 batch={'x': tensor([34, 33]), 'y': tensor([68, 66])} lr=['1.667e-03']
[2023-07-21 11:14:43,478 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=44.257 weight=0.753 batch=

### Learning Rate Scaler

In [27]:
lr_scaler_callback_demo = iter_based_trainer_demo.copy()
lr_scaler_callback = todd.Config(
    type='LRScaleCallback',
    lr_scaler=dict(base_batch_size=1),
)
lr_scaler_callback_demo.callbacks = [
    optimize_callback,
    lr_scaler_callback,
    log_callback,
]

In [28]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_scaler_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-21 11:14:43,588 27580:140704357856832][lr.py:92 todd.CustomIterBasedTrainer.custom_iter_based_trainer _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-07-21 11:14:43,600 27580:140704357856832][runners.py:61 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:43,604 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=18.593 weight=1.445 batch={'x': tensor([47, 20]), 'y': tensor([94, 40])}
[2023-07-21 11:14:43,608 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=0.245 weight=1.990 batch={'x': tensor([ 3, 46]), 'y': tensor([ 6, 92])}
[2023-07-21 11:14:43,613 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=16.500 weight=2.330 batch={'x': tensor([59, 41]

### Checkpoint

In [29]:
checkpoint_callback_demo = iter_based_trainer_demo.copy()
checkpoint_callback = todd.Config(type='CheckpointCallback', interval=10)
checkpoint_callback_demo.callbacks = [checkpoint_callback, log_callback]

In [30]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_50 = pathlib.Path(work_dirs) / 'custom_iter_based_trainer' / 'checkpoints' / 'iter_50'
    for f in iter_50.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-07-21 11:14:43,660 27580:140704357856832][runners.py:61 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:43,664 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=95.000 weight=0.000 batch={'x': tensor([59, 36]), 'y': tensor([118,  72])}
[2023-07-21 11:14:43,667 27580:140704357856832][checkpoint.py:59 todd.CustomIterBasedTrainer.custom_iter_based_trainer _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmposrv71ps/custom_iter_based_trainer/checkpoints/iter_10
[2023-07-21 11:14:43,670 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=124.000 weight=0.000 batch={'x': tensor([63, 61]), 'y': tensor([126, 122])}
[2023-07-21 11:14:43,673 27580:140704357856832][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trai


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmposrv71ps
└── custom_iter_based_trainer
    ├── 2023-07-21T11-14-43_660246-08-00.log
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_20
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_30
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_40
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_50
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├─

In [31]:
checkpoint_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
checkpoint_by_epoch_callback = checkpoint_callback.copy()
checkpoint_by_epoch_callback.update(interval=1, by_epoch=True)
checkpoint_by_epoch_callback_demo.callbacks = [
    checkpoint_by_epoch_callback,
    log_callback,
]

In [32]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_by_epoch_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'custom_epoch_based_trainer' / 'checkpoints' / 'epoch_2'
    for f in epoch_2.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-07-21 11:14:44,207 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:44,210 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-21 11:14:44,217 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=53.000 weight=0.000 batch={'x': tensor([39, 14]), 'y': tensor([78, 28])}
[2023-07-21 11:14:44,221 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=27.000 weight=0.000 batch={'x': tensor([26,  1]), 'y': tensor([52,  2])}
[2023-07-21 11:14:44,224 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=91.000 weight=0.000 batch={'x': tensor([54, 37]), 'y': tenso


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpj92b9ere
└── custom_epoch_based_trainer
    ├── 2023-07-21T11-14-44_206649-08-00.log
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├── meta.pth
            ├── model.pth
            ├── optim.pth
            └── strategy.pth

7 directories, 21 files

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                 

In [33]:
checkpoint_load_from_callback_demo = checkpoint_by_epoch_callback_demo.copy()
checkpoint_load_from_callback_demo.callbacks = [
    optimize_callback,
    checkpoint_by_epoch_callback,
    log_callback,
]

In [34]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
        load_from=os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2')
    )
    runner.run()

[2023-07-21 11:14:44,730 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:44,735 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-21 11:14:44,741 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=53.338 weight=0.745 batch={'x': tensor([32, 53]), 'y': tensor([ 64, 106])}
[2023-07-21 11:14:44,748 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=22.220 weight=1.560 batch={'x': tensor([41, 60]), 'y': tensor([ 82, 120])}


[2023-07-21 11:14:44,755 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=0.736 weight=2.047 batch={'x': tensor([ 8, 23]), 'y': tensor([16, 46])}
[2023-07-21 11:14:44,760 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [20/102] loss=0.094 weight=2.003 batch={'x': tensor([25, 50]), 'y': tensor([ 50, 100])}
[2023-07-21 11:14:44,766 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [25/102] loss=10.997 weight=1.793 batch={'x': tensor([49, 57]), 'y': tensor([ 98, 114])}
[2023-07-21 11:14:44,772 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [30/102] loss=3.604 weight=1.923 batch={'x': tensor([64, 29]), 'y': tensor([128,  58])}
[2023-07-21 11:14:44,784 27580:140704357856832][checkpoint.py:59 todd.CustomEpochBase


--------------------



[2023-07-21 11:14:45,325 27580:140704357856832][checkpoint.py:44 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer connect] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpw2_cbc5g/custom_epoch_based_trainer/checkpoints/epoch_2
[2023-07-21 11:14:45,328 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:45,329 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [3/3]
[2023-07-21 11:14:45,333 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [70/102] loss=8.759 weight=2.192 batch={'x': tensor([32, 59]), 'y': tensor([ 64, 118])}
[2023-07-21 11:14:45,336 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [75/102] loss=6.503 weight=2.128 batch={'x'

### Monitor

In [35]:
class CustomError(RuntimeError):
    pass

In [36]:
class FaultyRunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError('faulty runner')

In [37]:
@todd.RunnerRegistry.register()
class FaultyValidator(FaultyRunnerMixin, todd.runners.Validator):
    pass

In [38]:
@todd.RunnerRegistry.register()
class FaultyIterBasedTrainer(FaultyRunnerMixin, todd.runners.IterBasedTrainer):
    pass

In [39]:
@todd.RunnerRegistry.register()
class FaultyEpochBasedTrainer(
    FaultyRunnerMixin,
    todd.runners.EpochBasedTrainer,
):
    pass

In [40]:
monitor_callback_demo = validator_demo.copy()
monitor_callback_demo.type = 'FaultyValidator'
monitor_callback = todd.Config(type='MonitorCallback')
monitor_callback_demo.callbacks = [monitor_callback, log_callback]

In [41]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        monitor_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-07-21 11:14:45,431 27580:140704357856832][runners.py:61 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:45,433 27580:140704357856832][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x15760d4d0>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/runners.py", line 200, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_27580/2137902126.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner



[2023-07-21 11:14:45,431 27580:140704357856832][runners.py:61 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:45,433 27580:140704357856832][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x15760d4d0>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/runners.py", line 200, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_27580/2137902126.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner


### Priorities

## Strategies

In [42]:
strategy_load_model_from_demo = checkpoint_load_from_callback_demo.copy()

In [43]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.load_model_from(os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2', 'model.pth'))
    runner.run()

[2023-07-21 11:14:45,783 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:45,784 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-21 11:14:45,789 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=32.438 weight=0.702 batch={'x': tensor([ 5, 45]), 'y': tensor([10, 90])}
[2023-07-21 11:14:45,792 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=17.710 weight=1.615 batch={'x': tensor([43, 49]), 'y': tensor([86, 98])}
[2023-07-21 11:14:45,796 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=2.973 weight=2.145 batch={'x': tensor([ 3, 38]), 'y': tensor


--------------------



[2023-07-21 11:14:46,385 27580:140704357856832][runners.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-21 11:14:46,387 27580:140704357856832][base.py:60 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp116tf4dw/custom_epoch_based_trainer/checkpoints/epoch_2/model.pth
[2023-07-21 11:14:46,389 27580:140704357856832][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-21 11:14:46,393 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=7.525 weight=2.175 batch={'x': tensor([26, 60]), 'y': tensor([ 52, 120])}
[2023-07-21 11:14:46,396 27580:140704357856832][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=11.636 weigh

## Dry Run

In [44]:
todd.Store.DRY_RUN = True